# Thinking1:关联规则中的支持度、置信度和提升度代表的什么，如何计算

    
   |     | 概念  | 公式  |
   |  ----  | ----  |  ----  |
   | 支持度  | 商品在指定交易集合中出现的次数 |商品次数/总交易次数 |
   | 置信度  | 在商品A出现的情况下，商品b出现的概率，也就是商品B在有商品A出现的交易中出现的次数 |  商品AB同时出现的次数/商品A出现的次数 |
   | 提升度  | 商品A和商品B同时出现时候，商品B对商品A的支持程度 |支持度/置信度 |
    

# Thinking2 关联规则与协同过滤的区别
    
     关联规则：是基于交易行为对物体固有的内部关系进行发掘，找出其频繁项集
     协同过滤：是基于用户的历史行为习惯进行分析，挖掘用户喜好来进行推荐

# Thinking3 为什么我们需要多种推荐算法

   各种推荐算法都有其优点以及适用场景，比如在电商系统中，对于一些冷启动的用户或这商品，可以通过关联规则方式进行前期推荐，等到有一定的用户行为或用户对商品的操作反馈信息后，可以使用协同过滤的方式推荐。通过关联规则可以发掘一些整体上的宏观规律性质的信息，协同过滤更具有个性化的需求


# Thinking4 关联规则中的最小支持度、最小置信度该如何确定
 
 通过经验数据来设置，比如交易数据量很大，此时每个商品在总商品中出现的次数都很低，如果设置的最小支持度或者最小置信度过大，则很可能筛选不出任何结果
 也可以将每个商品的支持度，置信度计算出来，从大到小排序，然后卡定符合业务需求的位置所对应的指出度或置信度为最小支持度或者最小置信度


# Thinking5  都有哪些常见的回归分析方法，评价指标是什么

  常见的回归分析方法有：一元线性回归，多元线性回归 非线性回归
  评价指标是通过损失函数来评价的，有mse(更容易收敛)  MAE(可解释性更强) RMSE(既收敛又兼具可解释性)  R方值


# Action1  针对MarketBasket数据集进行购物篮分析（频繁项集及关联规则挖掘）[kaggle](https://www.kaggle.com/dragonheir/basket-optimisation)
         

In [50]:
'''
通过mlxtend的apriori方式对数据进行分析
'''

import pandas as pd
import fptools as fp
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
# 读取数据
datas=pd.read_csv('data/Market_Basket_Optimisation.csv',header=None)
# 通过迭代器方式对pd读取出来的数据转成list数组，并且出去其中的nan
itemsets = [[y for y in x if pd.notna(y)] for x in datas.values.tolist()]
# itemDits={}
# for items in itemsets:
#     for item in items:
#         if item not in itemDits.keys():
#             itemDits[item] = 1
#         else:
#             itemDits[item] = itemDits[item] + 1
# itemDits = sorted(itemDits.items(), key=lambda item:item[1], reverse=True)

# 转换为算法可接受模型（布尔值）one-hot操作
te = TransactionEncoder()
df_tf = te.fit_transform(itemsets)
df = pd.DataFrame(df_tf, columns=te.columns_)

# 设置支持度求频繁项集
frequent_itemsets = apriori(df, min_support=0.02, use_colnames=True)
# 求关联规则,设置最小置信度为0.15
print(frequent_itemsets)
rules = association_rules(frequent_itemsets,metric='confidence',  min_threshold=0.15)
# 设置最小提升度
rules = rules.drop(rules[rules.lift < 1.0].index)
# # 设置标题索引并打印结果
rules.rename(columns={'antecedents': 'from', 'consequents': 'to', 'support': 'sup', 'confidence': 'conf'}, inplace=True)
rules = rules[['from', 'to', 'sup', 'conf', 'lift']]
print(rules)



      support                           itemsets
0    0.020397                          (almonds)
1    0.033329                          (avocado)
2    0.033729                         (brownies)
3    0.087188                          (burgers)
4    0.030129                           (butter)
..        ...                                ...
98   0.020131  (mineral water, whole wheat rice)
99   0.022930             (spaghetti, olive oil)
100  0.025197              (spaghetti, pancakes)
101  0.021197                (spaghetti, shrimp)
102  0.020931              (tomatoes, spaghetti)

[103 rows x 2 columns]
                  from               to       sup      conf      lift
0               (eggs)        (burgers)  0.028796  0.160237  1.837830
1            (burgers)           (eggs)  0.028796  0.330275  1.837830
2            (burgers)   (french fries)  0.021997  0.252294  1.476173
3            (burgers)  (mineral water)  0.024397  0.279817  1.173883
4            (burgers)      (spaghetti